In [26]:
import ocr
import os
import numpy as np
#from collections import defaultdict

In [27]:
def ocr_folder(ocrspace, img_path):
    '''
    :type ocrspace: ocr.OCRSpace
    :type img_path: str
    :rtype: list
    '''
    ocr_result = []
    #dct_result = defaultdict(list)
    for idx, img_name in enumerate(sorted(os.listdir(img_path))):
        if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
            continue
        print('processing file {}'.format(img_name))
        img_file = os.path.join(img_path, img_name)
        file_ocr = ocrspace.ocr_file(img_file)
        n = len(file_ocr['ParsedResults'][0]['TextOverlay']['Lines'])
        for i in range(n):
            #words = file_ocr['ParsedResults'][0]['TextOverlay']['Lines'][i]['Words']
            #for j in range(len(words)):
            #   ocr_result.append(words[j]['WordText'])
            ocr_result.append(file_ocr['ParsedResults'][0]['TextOverlay']['Lines'][i]['Words'][-1]['WordText'])
            #dct_result[img_name[:4]].append(file_ocr['ParsedResults'][0]['TextOverlay']['Lines'][i]['Words'][-1]['WordText'])
            
    return ocr_result

def clean_with_heads_define(raw_result, heads):
    '''
    :type raw_result: list
    :type heads: dict
    :rtype: list
    '''
    ret = []
    i = 0
    while i < len(raw_result):
        if raw_result[i] in heads:
            if heads[raw_result[i]]:
                ret += raw_result[i+1:i+6]
            i += 5
        else:
            i += 1
    return ret
    
def clean(raw_result):
    heads = {  # for Numbers
             'Remaining':True, 'Limit':False, 'Thru':False, 'Number':True, 'Type':False, \
             'Date':True, 'Merchant':False, 'Amount':True, 'Status':False, 'Num': True, 'Numt': True, 'Numb':True # for Purchases
            }
    return clean_with_heads_define(raw_result, heads)

def card_trans(cardno, ocr_tran, save=False):
    trans = clean(ocr_tran)
    n = len(trans)
    print(len(trans))
    col, row = 3,5 
    np_trans = np.reshape(trans, (n//(col*row), col, row)).swapaxes(1,2).reshape(n//col, col)
    #print(np_trans_4804)
    print(np_trans.shape)
    # append a column with cardno
    new_col = np.full((np_trans.shape[0], 1), cardno, dtype='a4')
    np_trans = np.hstack((np_trans, new_col))
    if save:
        np.savetxt("/Users/peli/OneDrive/VCC/Aug11/trans{}.csv".format(cardno), np_trans, fmt="%s", delimiter=",")
    return np_trans

#print(clean(ocr_matrix2))
ocr_api = ocr.OCRSpace('6779f6135188957')

In [28]:
root_folder = "/Users/peli/OneDrive/VCC/Aug13/"
total_result = np.ndarray(shape=(0,4))
for f in sorted(os.listdir(root_folder)):
    if os.path.isdir(os.path.join(root_folder, f)) and (f in ['4804','5824', '9557', '4284']):
        print("processing folder:{}".format(f))
        ocr_tran = ocr_folder(ocr_api, os.path.join(root_folder, f))
        #print(len(ocr_tran))
        np_tran = card_trans(f, ocr_tran)
        #print(np_tran.shape)
        total_result = np.vstack((total_result, np_tran))
        print("current total trans:{}".format(total_result.shape))
np.savetxt("{}/trans.csv".format(root_folder), total_result, fmt="%s", delimiter=",")

processing folder:4284
processing file Screen Shot 2017-08-13 at 10.11.20 PM.png
15
(5, 3)
current total trans:(5, 4)
processing folder:4804
processing file Screen Shot 2017-08-13 at 10.09.54 PM.png
processing file Screen Shot 2017-08-13 at 10.10.00 PM.png
processing file Screen Shot 2017-08-13 at 10.10.08 PM.png
45
(15, 3)
current total trans:(20, 4)
processing folder:5824
processing file Screen Shot 2017-08-13 at 10.07.56 PM.png
processing file Screen Shot 2017-08-13 at 10.08.04 PM.png
30
(10, 3)
current total trans:(30, 4)
processing folder:9557
processing file Screen Shot 2017-08-13 at 10.10.42 PM.png
processing file Screen Shot 2017-08-13 at 10.10.50 PM.png
30
(10, 3)
current total trans:(40, 4)
